### Basic upload preparation

This notebook is a walk-through for uploading a basic network to the VR platform.

It starts by generating a random graph using the networkx module and converts it into a JSON file required by the VR platform.




In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import itertools
import numpy as np
import json
import random as rd

import plotly.graph_objects as go

**Welcome to the "Basic Upload Preparation" notebook!**

In this notebook, we will demonstrate the upload functionality for our VR (Virtual Reality) platform. The primary objective is to provide you with a step-by-step guide on how to prepare and upload content to the VR platform.

We will utilize the networkx module to generate a random graph. The graph's node and edge structure will serve as an example dataset for our demonstration. We will then convert this data into a JSON file, which is the required format for uploading content to the VR platform.

In this notebook, we will cover the following key topics:

1. Generate a Random Graph: We use the networkx module to create a random graph with nodes and edges. This graph will serve as our sample dataset.

2. Preview the Graph: We use plotly to preview our graph to verify our network.

3. Data Preprocessing: We walk through preprocessing steps to transform the graph data into a format suitable for the VR platform. This includes converting the node and edge structure into a JSON file.


In [3]:
# # For running in colab directly: mount google drive so we can save files

# from google.colab import drive
# drive.mount('/content/gdrive')
# root = '/content/gdrive/'

# # change this if you'd like to save somewhere else in your google drive!
# save_prefix = root + 'MyDrive/Colab Notebooks/'
save_prefix = ''
fn_json = save_prefix + 'BA_basic.json'

### Barabasi-Albert random network

The Barabasi-Albert model uses a preferential attachment model to generate a random network with a power-law degree distribution.

It starts with a (small) number of nodes, $m$.  New nodes ($n$ of them) are added one at a time, making connections to existing nodes with probability proportional to the existing node's degree.  That means, an existing node with lots of connections is likely to connect to the new node; and an existing node with very few connections is unlikely to make new connections.

This process results in a network where most nodes have just a few connections, and a handful of nodes have a huge number of connections.


In [4]:
# Create a Barabasi-Albert network

m = 1
n = 100

G = nx.barabasi_albert_graph(n, m)

### layout

We will use the spring layout to position the nodes into 3D space.

The spring layout is one of the most popular layouts, but for larger networks it often results in a not very helpful "hairball".  We'll show you some alternative network layouts later!

In [5]:
# Generate three-dimensional coordinates

# random positions
pos3D_rnd = nx.random_layout(G, dim=3)

# spring- (organic-) layout
pos3D_spg = nx.spring_layout(G, dim=3)



### Preview the graph

Now, we have our Albert-Barabasi network stored in our variable ```G```, and the positions of our nodes stored in ```pos3D_rnd``` and ```pos3D_spg```.

Let's check it out quickly with plotly before we put it into the VR platform.  The next block defines a function for our quick visualization.

In [6]:
def plotly_preview(G, pos, node_colors=None, edge_colors=None):

    """
    Generate a 3D network visualization using Plotly.

    Parameters:
        - G (networkx.Graph): The graph object representing the network.
        - pos3D (dict): A dictionary mapping each node to its 3D coordinates (x, y, z).
        - node_colors (dict, optional): A dictionary mapping nodes to custom hex colors.
                                        If not provided, the default color is '#40b9d4'.
        - edge_colors (dict, optional): A dictionary mapping edges to custom hex colors.
                                        If not provided, the default color is 'gray'.

    Output:
        - An HTML file named 'network_visualization.html' is generated, which opens in a web browser.

    Example usage:
        node_colors = {1: '#ff0000', 2: '#00ff00', 3: '#0000ff'}
        edge_colors = {(1, 2): '#ff00ff', (2, 3): '#ffff00'}
        pos = nx.spring_layout(G,dim=3)
        plotly_review_2(G, pos, node_colors, edge_colors)
    """

    # Create a Plotly figure
    fig = go.Figure()

    # Add nodes to the figure
    for node in G.nodes():
        x, y, z = pos[node]
        color = node_colors[node] if node_colors and node in node_colors else '#40b9d4'
        fig.add_trace(go.Scatter3d(
            x=[x],
            y=[y],
            z=[z],
            mode='markers',
            marker=dict(
                size=5,
                color=color,
            ),
            name=str(node),
            text=str(node),
            hovertemplate=None,
        ))

    # Add edges to the figure
    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_color = edge_colors[edge] if edge_colors and edge in edge_colors else 'gray'
        fig.add_trace(go.Scatter3d(
            x=[x0, x1],
            y=[y0, y1],
            z=[z0, z1],
            mode='lines',
            line=dict(
                color=edge_color,
                width=1,
            ),
            hoverinfo='none',
        ))

    # Set layout options
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(l=0, r=0, b=0, t=0),
    )

    # Save the figure as an HTML file
    fig.write_html('network_preview.html', auto_open=True)

Finally, let's see our network!

In [7]:
plotly_preview(G,pos3D_spg)

### prepare VR upload

For the VR upload, we need to specify some more properties of our network:
* Node positions
* Node colors
* Edge colors
* Node annotations
* Communities of nodes (optional)

Below, we walk through the steps of storing these properties in the networkx graph structure, and as a last step, we convert the networkx graph object into a JSON file, which can be used with the VR uploader.


In [ ]:

def make_json(name, network, positions, node_color = '#40b9d4', link_color = '#999999', annotations = 'None', communities = 'None'):
    """
    Generates a JSON file from a given network graph using the specified parameters.

    Args:
        name (str, optional): Name of the graph.
        network (networkx.Graph): Network graph object.
        positions (dict): Dictionary mapping node IDs to their positions.
        node_color (dict): Dictionary mapping node IDs to their (hex-)colors.
        link_color (str or dict): (Hex-)color value for all links in the graph or dict with node tuple as key and hex color as value.
        communities (dict): 'None' for no communities (default) or dictionary mapping node IDs to their corresponding community ID.
        annotations (dict): Dictionary mapping node IDs to a list of annotations.

    Returns:
        None

    """

    # --------------------------
    # Generate VR GRAPH
    # --------------------------
    GVR = nx.Graph()
    GVR.graph['name'] = name

    # --------------------------------------
    # LOOKUP FOR NODE NAMES INTO IDs and vv
    # --------------------------------------
    d_idx_node = {}
    d_node_idx = {}
    for i, node in enumerate(sorted(network.nodes())):
        d_idx_node[i] = node
        d_node_idx[node] = i
    GVR.add_nodes_from(d_idx_node.keys())

    for edge in network.edges()(data=True):
        GVR.add_edge(d_node_idx[edge[0]],d_node_idx[edge[1]])

    # --------------------------
    # POS
    # --------------------------
    if isinstance(positions[next(iter(positions))], list):
        pass
    else:
        for key in positions:
            positions[key] = positions[key].tolist()

    posG = {d_node_idx[node]: list(xyz) for node, xyz in positions.items()}
    nx.set_node_attributes(GVR, posG, name="pos")

    # # --------------------------
    # # CLUSTER
    # # --------------------------
    if communities == 'None':
        dict_for_cluster = dict(zip(d_idx_node.keys(), [0 for _ in d_idx_node.keys()]))
    else:
        d_VRids_cluster = {d_node_idx[node]: str(cl_id) for node, cl_id in communities.items()}
        nx.set_node_attributes(GVR, d_VRids_cluster, name="cluster")


    # --------------------------
    # NODE COLOR
    # --------------------------
    d_node_colors={}

    if isinstance(node_color, dict):
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color[d_idx_node[nodeid]]
    else:
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color

    nx.set_node_attributes(GVR, d_node_colors, name="nodecolor")

    # --------------------------
    # LINK COLOR
    # --------------------------
    if isinstance(link_color, dict):
        # for different link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            try:
                color = link_color[(d_idx_node[a],d_idx_node[b])]
            except KeyError:
                color = link_color[(d_idx_node[b],d_idx_node[a])]
            d_edge_color[(a,b)] = color
    else:
        # for unique link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            d_edge_color[(a,b)] = link_color

    nx.set_edge_attributes(GVR, d_edge_color, name="linkcolor")

    # --------------------------
    # NODE ANNOTATION
    # --------------------------
    if isinstance(annotations, dict):

        l_annotations = [[str(d_idx_node[nodeid])] + [ annotation for annotation in annotations[d_idx_node[nodeid]]] for nodeid in sorted(GVR.nodes())]
        d_annotations = dict(zip(sorted(GVR.nodes()), l_annotations))
    else:
        d_annotations = {nodeid: [str(d_idx_node[nodeid])] for nodeid in GVR.nodes()}

    nx.set_node_attributes(GVR, d_annotations, name="annotation")

    # --------------------------
    # MAKE JSON fo uploader
    # --------------------------

    G_json = json.dumps(nx.node_link_data(GVR))

    with open(GVR.name, "w") as outfile:
        outfile.write(G_json)


Make JSON files required for the VR uploader

In [ ]:
make_json(name = fn_json, network=G, positions=pos3D_spg)